In [ ]:
import cv2
import numpy as np
import os
import match_images
# from create_projected_gaze import projected_gaze
from ptsInteretFixations import *
from appelsDB import load_from_db
from heat_map import *
from create_projected_gaze import projected_gaze
from convert_to_sql import csv_to_sqlite

WORKING_DIR = "data"
SUJET_NAMES = ["sujet1_f-42e0d11a", "sujet2_f-835bf855", "sujet3_m-84ce1158", "sujet4_m-fee537df", "sujet5_m-671cf44e", "sujet6_m-0b355b51"]
VIDEO_FILENAMES = ["e0b2c246_0.0-138.011.mp4", "b7bd6c34_0.0-271.583.mp4", "422f10f2_0.0-247.734.mp4", "2fb8301a_0.0-71.632.mp4", "585d8df7_0.0-229.268.mp4", "429d311a_0.0-267.743.mp4"]
POSTERS_DIR = f"{WORKING_DIR}/Affiches"


In [ ]:
images = []
sujet_id = 0  # Choisir le sujet (0 à 5)
DB_PATH = f"{WORKING_DIR}/database{sujet_id+1}.sqlite"
positions_all_posters = dict() # le resultat final

# Création de la DB SQLite si elle n'existe pas déjà
if not os.path.exists(DB_PATH):
    print(f"[INFO] Création de la DB SQLite pour le sujet {sujet_id+1}...")
    csv_to_sqlite(f"{WORKING_DIR}/{SUJET_NAMES[sujet_id]}", DB_PATH, display)

for filename in os.listdir(POSTERS_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(POSTERS_DIR, filename)
        img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            positions_all_posters[filename] = ([], [])  # Initialize lists for x and y positions



res = ORB_on_fixations(f"./{WORKING_DIR}/{SUJET_NAMES[sujet_id]}", db_path=DB_PATH, video_filename=f"{VIDEO_FILENAMES[sujet_id]}")
print("\nNombre de fixations:", len(res))

In [ ]:
print(positions_all_posters)

In [ ]:
res_out = res
n = 3000
kp_desc_all_posters = [match_images.apply_orb(img,n) for img in images]
seuil = 20
for entry in res_out:
    fix_index = entry["fix_index"]
    image_test = entry["frame"]
    h, w = image_test.shape[:2]
    id_best_match, H = match_images.match_all(images,image_test,kp_desc_all_posters,seuil,n)
    print("Fixation " + str(fix_index) + ", poster retrouvé:", os.listdir(POSTERS_DIR)[id_best_match] if id_best_match!=-1 else "Aucun")

    if id_best_match != -1:
        # load gazes from DB which match the fixation id in res[fix_index]
        arr = load_from_db(
            db_path=DB_PATH,
            cols=["gaze x [px]", "gaze y [px]"],
            table="gaze",
            where_clause='"fixation id" = ?',
            where_params=[fix_index],
        )
        arr = projected_gaze(arr, H, clip=False, poster_size=(images[id_best_match].shape[1], images[id_best_match].shape[0]))
        
        if len(arr) != 0:
            h, w = images[id_best_match].shape[:2]
            x = arr[:,0]
            y = arr[:,1]
            y_plotly = h - y

            positions_all_posters[os.listdir(POSTERS_DIR)[id_best_match]][0].extend(x.tolist())
            positions_all_posters[os.listdir(POSTERS_DIR)[id_best_match]][1].extend(y_plotly.tolist())
        else :
            print("Aucun point de regard projeté sur le poster.")


In [ ]:
# afficher les points pour chaque poster
for poster_name, (x, y) in positions_all_posters.items():
    if len(x) > 1 and len(y) > 1:
        step_heat_map(x, y, poster_name, trace=False)